In [110]:
import json
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [111]:
df_wind = pd.read_csv("us_wind.csv", sep = ',')
df_wind

,case_id,faa_ors,faa_asn,usgs_pr_id,t_state,t_county,t_fips,p_name,p_year,p_tnum,...,t_hh,t_rd,t_rsa,t_ttlh,t_conf_atr,t_conf_loc,t_img_date,t_img_srce,xlong,ylat
0,3073429,missing,missing,4960,CA,Kern County,6029,251 Wind,1987,194,...,-9999.0,-9999.0,-9999.0,-9999.0,2,3,1/1/2012,NAIP,-118.360725,35.083778
1,3071522,missing,missing,4997,CA,Kern County,6029,251 Wind,1987,194,...,-9999.0,-9999.0,-9999.0,-9999.0,2,3,1/1/2012,NAIP,-118.361168,35.081512
2,3073425,missing,missing,4957,CA,Kern County,6029,251 Wind,1987,194,...,-9999.0,-9999.0,-9999.0,-9999.0,2,3,1/1/2012,NAIP,-118.360420,35.084709
3,3071569,missing,missing,5023,CA,Kern County,6029,251 Wind,1987,194,...,-9999.0,-9999.0,-9999.0,-9999.0,2,3,7/31/2016,Digital Globe,-118.364029,35.079418
4,3005252,missing,missing,5768,CA,Kern County,6029,251 Wind,1987,194,...,-9999.0,-9999.0,-9999.0,-9999.0,2,3,11/23/2017,Digital Globe,-118.354286,35.085594
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58180,3040109,06-037711,2011-WTW-12036-OE,-9999,CA,Yolo County,6113,unknown Yolo County,2014,1,...,-9999.0,-9999.0,-9999.0,28.7,1,3,6/18/2017,Digital Globe,-121.861214,38.580673
58181,3040944,48-024978,2011-WTW-352-OE,41364,TX,Young County,48503,unknown Young County 1,2011,1,...,-9999.0,-9999.0,-9999.0,29.0,1,3,missing,Digital Globe,-98.551094,33.093292
58182,3041125,48-025943,2011-WTW-4884-OE,41365,TX,Young County,48503,unknown Young County 2,2011,1,...,-9999.0,-9999.0,-9999.0,34.1,1,3,missing,Digital Globe,-98.617889,33.094593
58183,3055917,08-072245,2015-WTW-9994-OE,-9999,CO,Yuma County,8125,unknown Yuma County,2016,2,...,-9999.0,-9999.0,-9999.0,48.8,1,3,5/17/2017,Digital Globe,-102.717171,40.035072


In [112]:
df_wind = df_wind.drop( ["t_hh","case_id","t_rd","t_rsa", "p_cap", "t_conf_atr", "t_conf_loc","t_fips", "t_img_date", "t_img_srce", "faa_ors","faa_asn", "usgs_pr_id"], axis=1 )

In [113]:
df_wind

,t_state,t_county,p_name,p_year,p_tnum,t_manu,t_model,t_cap,t_ttlh,xlong,ylat
0,CA,Kern County,251 Wind,1987,194,Vestas,missing,95,-9999.0,-118.360725,35.083778
1,CA,Kern County,251 Wind,1987,194,Vestas,missing,95,-9999.0,-118.361168,35.081512
2,CA,Kern County,251 Wind,1987,194,Vestas,missing,95,-9999.0,-118.360420,35.084709
3,CA,Kern County,251 Wind,1987,194,Vestas,missing,95,-9999.0,-118.364029,35.079418
4,CA,Kern County,251 Wind,1987,194,Vestas,missing,95,-9999.0,-118.354286,35.085594
...,...,...,...,...,...,...,...,...,...,...,...
58180,CA,Yolo County,unknown Yolo County,2014,1,missing,missing,-9999,28.7,-121.861214,38.580673
58181,TX,Young County,unknown Young County 1,2011,1,missing,missing,-9999,29.0,-98.551094,33.093292
58182,TX,Young County,unknown Young County 2,2011,1,missing,missing,-9999,34.1,-98.617889,33.094593
58183,CO,Yuma County,unknown Yuma County,2016,2,missing,missing,-9999,48.8,-102.717171,40.035072


## Dropping rows with missing data

Upon getting the data, we see that there are many missing values in the "t_manu", "t_model", "t_cap", "p_year" and "t_ttlh" columns

We then filter these and drop them from the dataFrame

### Drop missing "t_manu"

In [114]:
# Get the rows that have the missing "t_manu"
t_manu_drop = df_wind.index[df_wind['t_manu'] == 'missing'].tolist()

In [115]:
# Now drop those rows from the dataFrame
df_wind.drop(t_manu_drop, axis=0, inplace=True)

In [116]:
df_wind

,t_state,t_county,p_name,p_year,p_tnum,t_manu,t_model,t_cap,t_ttlh,xlong,ylat
0,CA,Kern County,251 Wind,1987,194,Vestas,missing,95,-9999.0,-118.360725,35.083778
1,CA,Kern County,251 Wind,1987,194,Vestas,missing,95,-9999.0,-118.361168,35.081512
2,CA,Kern County,251 Wind,1987,194,Vestas,missing,95,-9999.0,-118.360420,35.084709
3,CA,Kern County,251 Wind,1987,194,Vestas,missing,95,-9999.0,-118.364029,35.079418
4,CA,Kern County,251 Wind,1987,194,Vestas,missing,95,-9999.0,-118.354286,35.085594
...,...,...,...,...,...,...,...,...,...,...,...
55167,TX,El Paso County,unknown El Paso County,2015,2,GE Wind,GE1.7-100,1700,130.1,-106.405434,31.788124
55168,TX,El Paso County,unknown El Paso County,2015,2,GE Wind,GE1.7-100,1700,130.1,-106.405670,31.788097
55368,NY,Erie County,unknown Erie County,2016,2,Northern Power Systems,NPS-100,100,47.5,-78.931122,42.977005
55531,AK,Northwest Arctic Borough,unknown Kotzebue Wind Project,2003,2,Seaforth Energy,AOC15_65,65,38.0,-162.551575,66.837898


### Drop missing "t_model"

In [117]:
# Get the rows that have the missing "t_model"
t_model_drop = df_wind.index[df_wind['t_model'] == 'missing'].tolist()

In [118]:
# Now drop those rows from the dataFrame
df_wind.drop(t_model_drop, axis=0, inplace=True)

In [119]:
df_wind

,t_state,t_county,p_name,p_year,p_tnum,t_manu,t_model,t_cap,t_ttlh,xlong,ylat
190,IA,Story County,30 MW Iowa DG Portfolio,2017,10,Nordex,AW125/3000,3000,150.0,-93.632835,41.882477
191,IA,Story County,30 MW Iowa DG Portfolio,2017,10,Nordex,AW125/3000,3000,150.0,-93.623009,41.881470
192,IA,Story County,30 MW Iowa DG Portfolio,2017,10,Nordex,AW125/3000,3000,150.0,-93.513710,42.019119
193,IA,Story County,30 MW Iowa DG Portfolio,2017,10,Nordex,AW125/3000,3000,150.0,-93.430367,42.028233
194,IA,Story County,30 MW Iowa DG Portfolio,2017,10,Nordex,AW125/3000,3000,150.0,-93.518082,42.013630
...,...,...,...,...,...,...,...,...,...,...,...
55167,TX,El Paso County,unknown El Paso County,2015,2,GE Wind,GE1.7-100,1700,130.1,-106.405434,31.788124
55168,TX,El Paso County,unknown El Paso County,2015,2,GE Wind,GE1.7-100,1700,130.1,-106.405670,31.788097
55368,NY,Erie County,unknown Erie County,2016,2,Northern Power Systems,NPS-100,100,47.5,-78.931122,42.977005
55531,AK,Northwest Arctic Borough,unknown Kotzebue Wind Project,2003,2,Seaforth Energy,AOC15_65,65,38.0,-162.551575,66.837898


### Drop missing "t_cap"

In [120]:
# Get the rows that have the missing "t_cap"
t_cap_drop = df_wind.index[df_wind['t_cap'] == -9999].tolist()

In [121]:
# Now drop those rows from the dataFrame
df_wind.drop(t_cap_drop, axis=0, inplace=True)

In [122]:
df_wind

,t_state,t_county,p_name,p_year,p_tnum,t_manu,t_model,t_cap,t_ttlh,xlong,ylat
190,IA,Story County,30 MW Iowa DG Portfolio,2017,10,Nordex,AW125/3000,3000,150.0,-93.632835,41.882477
191,IA,Story County,30 MW Iowa DG Portfolio,2017,10,Nordex,AW125/3000,3000,150.0,-93.623009,41.881470
192,IA,Story County,30 MW Iowa DG Portfolio,2017,10,Nordex,AW125/3000,3000,150.0,-93.513710,42.019119
193,IA,Story County,30 MW Iowa DG Portfolio,2017,10,Nordex,AW125/3000,3000,150.0,-93.430367,42.028233
194,IA,Story County,30 MW Iowa DG Portfolio,2017,10,Nordex,AW125/3000,3000,150.0,-93.518082,42.013630
...,...,...,...,...,...,...,...,...,...,...,...
55167,TX,El Paso County,unknown El Paso County,2015,2,GE Wind,GE1.7-100,1700,130.1,-106.405434,31.788124
55168,TX,El Paso County,unknown El Paso County,2015,2,GE Wind,GE1.7-100,1700,130.1,-106.405670,31.788097
55368,NY,Erie County,unknown Erie County,2016,2,Northern Power Systems,NPS-100,100,47.5,-78.931122,42.977005
55531,AK,Northwest Arctic Borough,unknown Kotzebue Wind Project,2003,2,Seaforth Energy,AOC15_65,65,38.0,-162.551575,66.837898


### Drop missing "t_ttlh"

In [123]:
# Get the rows that have the missing "t_ttlh"
t_ttlh_drop = df_wind.index[df_wind['t_ttlh'] == -9999].tolist()

In [124]:
# Now drop those rows from the dataFrame
df_wind.drop(t_ttlh_drop, axis=0, inplace=True)

In [125]:
df_wind

,t_state,t_county,p_name,p_year,p_tnum,t_manu,t_model,t_cap,t_ttlh,xlong,ylat
190,IA,Story County,30 MW Iowa DG Portfolio,2017,10,Nordex,AW125/3000,3000,150.0,-93.632835,41.882477
191,IA,Story County,30 MW Iowa DG Portfolio,2017,10,Nordex,AW125/3000,3000,150.0,-93.623009,41.881470
192,IA,Story County,30 MW Iowa DG Portfolio,2017,10,Nordex,AW125/3000,3000,150.0,-93.513710,42.019119
193,IA,Story County,30 MW Iowa DG Portfolio,2017,10,Nordex,AW125/3000,3000,150.0,-93.430367,42.028233
194,IA,Story County,30 MW Iowa DG Portfolio,2017,10,Nordex,AW125/3000,3000,150.0,-93.518082,42.013630
...,...,...,...,...,...,...,...,...,...,...,...
55167,TX,El Paso County,unknown El Paso County,2015,2,GE Wind,GE1.7-100,1700,130.1,-106.405434,31.788124
55168,TX,El Paso County,unknown El Paso County,2015,2,GE Wind,GE1.7-100,1700,130.1,-106.405670,31.788097
55368,NY,Erie County,unknown Erie County,2016,2,Northern Power Systems,NPS-100,100,47.5,-78.931122,42.977005
55531,AK,Northwest Arctic Borough,unknown Kotzebue Wind Project,2003,2,Seaforth Energy,AOC15_65,65,38.0,-162.551575,66.837898


## Drop missing p_year

In [126]:
# Get the rows that have the missing "p_year"
p_year_drop = df_wind.index[df_wind['p_year'] == -9999].tolist()

In [127]:
# Now drop those rows from the dataFrame
df_wind.drop(p_year_drop, axis=0, inplace=True)

In [128]:
df_wind

,t_state,t_county,p_name,p_year,p_tnum,t_manu,t_model,t_cap,t_ttlh,xlong,ylat
190,IA,Story County,30 MW Iowa DG Portfolio,2017,10,Nordex,AW125/3000,3000,150.0,-93.632835,41.882477
191,IA,Story County,30 MW Iowa DG Portfolio,2017,10,Nordex,AW125/3000,3000,150.0,-93.623009,41.881470
192,IA,Story County,30 MW Iowa DG Portfolio,2017,10,Nordex,AW125/3000,3000,150.0,-93.513710,42.019119
193,IA,Story County,30 MW Iowa DG Portfolio,2017,10,Nordex,AW125/3000,3000,150.0,-93.430367,42.028233
194,IA,Story County,30 MW Iowa DG Portfolio,2017,10,Nordex,AW125/3000,3000,150.0,-93.518082,42.013630
...,...,...,...,...,...,...,...,...,...,...,...
55167,TX,El Paso County,unknown El Paso County,2015,2,GE Wind,GE1.7-100,1700,130.1,-106.405434,31.788124
55168,TX,El Paso County,unknown El Paso County,2015,2,GE Wind,GE1.7-100,1700,130.1,-106.405670,31.788097
55368,NY,Erie County,unknown Erie County,2016,2,Northern Power Systems,NPS-100,100,47.5,-78.931122,42.977005
55531,AK,Northwest Arctic Borough,unknown Kotzebue Wind Project,2003,2,Seaforth Energy,AOC15_65,65,38.0,-162.551575,66.837898


## Adding a new column

We'll be adding a new column <b>range</b> to the data.
This column would specify the ranges values in the "t_cap" column

The ranges would be:

<li> < 1000</li>
<li>1000 - 2000</li>
<li>2000 - 3000</li>
<li> > 3000</li>

In [129]:
# Get the "t_cap" column:
t_cap = df_wind['t_cap']

In [130]:
range_list = []

In [131]:
for value in t_cap:
    if value < 1000:
        range_list.append("< 1000")
    elif value >= 1000 and value < 2000:
        range_list.append("1000 - 2000")
    elif value >= 2000 and value < 3000:
        range_list.append("2000 - 3000")
    elif value >= 3000:
        range_list.append("≥ 3000")

In [132]:
# Adding the list as a new column
df_wind['Range'] = range_list

In [133]:
df_wind

,t_state,t_county,p_name,p_year,p_tnum,t_manu,t_model,t_cap,t_ttlh,xlong,ylat,Range
190,IA,Story County,30 MW Iowa DG Portfolio,2017,10,Nordex,AW125/3000,3000,150.0,-93.632835,41.882477,≥ 3000
191,IA,Story County,30 MW Iowa DG Portfolio,2017,10,Nordex,AW125/3000,3000,150.0,-93.623009,41.881470,≥ 3000
192,IA,Story County,30 MW Iowa DG Portfolio,2017,10,Nordex,AW125/3000,3000,150.0,-93.513710,42.019119,≥ 3000
193,IA,Story County,30 MW Iowa DG Portfolio,2017,10,Nordex,AW125/3000,3000,150.0,-93.430367,42.028233,≥ 3000
194,IA,Story County,30 MW Iowa DG Portfolio,2017,10,Nordex,AW125/3000,3000,150.0,-93.518082,42.013630,≥ 3000
...,...,...,...,...,...,...,...,...,...,...,...,...
55167,TX,El Paso County,unknown El Paso County,2015,2,GE Wind,GE1.7-100,1700,130.1,-106.405434,31.788124,1000 - 2000
55168,TX,El Paso County,unknown El Paso County,2015,2,GE Wind,GE1.7-100,1700,130.1,-106.405670,31.788097,1000 - 2000
55368,NY,Erie County,unknown Erie County,2016,2,Northern Power Systems,NPS-100,100,47.5,-78.931122,42.977005,< 1000
55531,AK,Northwest Arctic Borough,unknown Kotzebue Wind Project,2003,2,Seaforth Energy,AOC15_65,65,38.0,-162.551575,66.837898,< 1000


## Get all maximum and minimum values

We sought out to find the maximum values in all the quantitative data in the dataset. These values are then used to scale the plots properly

In [134]:
# Maximum and minimum p_year (Project year)
max_year = df_wind['p_year'].max()
min_year = df_wind['p_year'].min()
print("p_year: [%d, %d]" % (max_year, min_year))

p_year: [2018, 1983]


In [135]:
# Maximum and minimum p_tnum (number of turbines in project)
max_tnum = df_wind['p_tnum'].max()
min_tnum = df_wind['p_tnum'].min()
print("p_tnum: [%d, %d]" % (max_tnum, min_tnum))

p_tnum: [460, 1]


In [136]:
# Maximum and minimum t_cap ()
max_tcap = df_wind['t_cap'].max()
min_tcap = df_wind['t_cap'].min()
print("t_cap: [%d, %d]" % (max_tcap, min_tcap))

t_cap: [6000, 50]


In [137]:
# Maximum and minimum t_ttlh
max_ttlh = df_wind['t_ttlh'].max()
min_ttlh = df_wind['t_ttlh'].min()
print("t_ttlh: [%d, %d]" % (max_ttlh, min_ttlh))

t_ttlh: [198, 25]


## Export Files

In [138]:
df_wind.to_csv('wind_usa_cleaned.csv', index=False)

In [139]:
df_wind

,t_state,t_county,p_name,p_year,p_tnum,t_manu,t_model,t_cap,t_ttlh,xlong,ylat,Range
190,IA,Story County,30 MW Iowa DG Portfolio,2017,10,Nordex,AW125/3000,3000,150.0,-93.632835,41.882477,≥ 3000
191,IA,Story County,30 MW Iowa DG Portfolio,2017,10,Nordex,AW125/3000,3000,150.0,-93.623009,41.881470,≥ 3000
192,IA,Story County,30 MW Iowa DG Portfolio,2017,10,Nordex,AW125/3000,3000,150.0,-93.513710,42.019119,≥ 3000
193,IA,Story County,30 MW Iowa DG Portfolio,2017,10,Nordex,AW125/3000,3000,150.0,-93.430367,42.028233,≥ 3000
194,IA,Story County,30 MW Iowa DG Portfolio,2017,10,Nordex,AW125/3000,3000,150.0,-93.518082,42.013630,≥ 3000
...,...,...,...,...,...,...,...,...,...,...,...,...
55167,TX,El Paso County,unknown El Paso County,2015,2,GE Wind,GE1.7-100,1700,130.1,-106.405434,31.788124,1000 - 2000
55168,TX,El Paso County,unknown El Paso County,2015,2,GE Wind,GE1.7-100,1700,130.1,-106.405670,31.788097,1000 - 2000
55368,NY,Erie County,unknown Erie County,2016,2,Northern Power Systems,NPS-100,100,47.5,-78.931122,42.977005,< 1000
55531,AK,Northwest Arctic Borough,unknown Kotzebue Wind Project,2003,2,Seaforth Energy,AOC15_65,65,38.0,-162.551575,66.837898,< 1000


### Export smaller file for testing


In [140]:
import random
randomlist = []
for i in range(0,200):
    n = random.randint(0,58184)
    randomlist.append(n)
print(randomlist)

[18766, 39316, 7063, 45910, 50976, 36609, 41568, 4251, 33340, 55838, 54216, 35, 48207, 42283, 12046, 6546, 40236, 47530, 43078, 39697, 49347, 2807, 38584, 11380, 38463, 56535, 48336, 15709, 35236, 53650, 23035, 20236, 39144, 18714, 6847, 18560, 10798, 10723, 31820, 43905, 29244, 38041, 45365, 45882, 4362, 1117, 14051, 23797, 25588, 19808, 28494, 10951, 8876, 24759, 44302, 16559, 31136, 5844, 36509, 54658, 9679, 24600, 33411, 14594, 22961, 4563, 18373, 45069, 20771, 15585, 40979, 13224, 51901, 26546, 20430, 36119, 11432, 11042, 30520, 30686, 54438, 6752, 14917, 4497, 38890, 42941, 5906, 36909, 49860, 44751, 5373, 1201, 22769, 50342, 13143, 21162, 53858, 41611, 20442, 30479, 20304, 51132, 41086, 15560, 48257, 6766, 48831, 18937, 29581, 30026, 22207, 13184, 55865, 53617, 27336, 8789, 26148, 45636, 18649, 55016, 41126, 26009, 45121, 53719, 15630, 16625, 10849, 42899, 37334, 32183, 306, 31101, 1642, 11053, 33483, 38082, 47733, 55930, 51630, 10251, 9369, 34802, 31672, 29669, 20401, 16206, 33

In [141]:
df_wind_short=df_wind[df_wind.index.isin(randomlist)]
print(df_wind_short)

      t_state           t_county                    p_name  p_year  p_tnum  \
306        IA       Adams County                     Adams    2016      60   
1117       CA        Kern County                   Alta XI    2013      41   
1201       TX      Scurry County    Amazon Wind Farm Texas    2017     110   
1642       OK      Murray County         Arbuckle Mountain    2015      50   
2807       SD  Hutchinson County       Beethoven Wind, LLC    2015      43   
...       ...                ...                       ...     ...     ...   
53694      CA        Kern County                  Windstar    2012      23   
53719      CA        Kern County                  Windstar    2012      37   
53858      WA   Klickitat County            Windy Point II    2009      88   
54203      TX       Pecos County  Woodward Mountain I & II    2001     242   
54216      TX       Pecos County  Woodward Mountain I & II    2001     242   

        t_manu      t_model  t_cap  t_ttlh       xlong       yl

In [142]:
df_wind_short.to_csv('wind_usa_cleaned_st.csv', index=False)

In [143]:
import random
randomlist2 = []
for i in range(0,20000):
    n = random.randint(0,58184)
    randomlist2.append(n)
print(randomlist2)

[21431, 19341, 30152, 52304, 53514, 42429, 12280, 22631, 24966, 19836, 38241, 21528, 43824, 9283, 40816, 42818, 41661, 15280, 11600, 49820, 14759, 25243, 30949, 13897, 52533, 47818, 5972, 17927, 22276, 27784, 18464, 19531, 41915, 45463, 12150, 48817, 4587, 46239, 54248, 51672, 40967, 27792, 31145, 253, 13949, 6546, 43355, 36095, 48415, 52072, 42311, 16534, 714, 40157, 6091, 128, 24335, 57224, 14808, 7795, 12302, 35181, 18573, 34235, 18216, 22704, 171, 24792, 51966, 860, 1088, 24015, 49175, 42200, 47521, 17935, 29956, 1, 3736, 48343, 57481, 11606, 36426, 9555, 1381, 16050, 7477, 6447, 49426, 21762, 18918, 22765, 16471, 19004, 31667, 4173, 26499, 40754, 829, 24707, 38792, 6228, 46492, 23733, 1319, 377, 44587, 6135, 54144, 3538, 38742, 23736, 37797, 6213, 44588, 33140, 22068, 52042, 25284, 4297, 4062, 31222, 10272, 31556, 35021, 31029, 1136, 49960, 2135, 32584, 3277, 6060, 16535, 4386, 54258, 57486, 40475, 55901, 24145, 30721, 48724, 55873, 29923, 53894, 56419, 29070, 26396, 36902, 827, 7

In [108]:
df_wind_short2=df_wind[df_wind.index.isin(randomlist2)]
print(df_wind_short2)

      t_state                         p_name  p_year  p_tnum           t_manu  \
197        IA        30 MW Iowa DG Portfolio    2017      10           Nordex   
200        MA     6th Space Warning Squadron    2013       2          GE Wind   
202        MA             AFCEE MMR Turbines    2011       2          GE Wind   
203        MA             AFCEE MMR Turbines    2011       2          GE Wind   
205        IA                      AG Land 2    2012       1          GE Wind   
...       ...                            ...     ...     ...              ...   
54343      TX       Woodward Mountain I & II    2001     242           Vestas   
54353      ID                    Yahoo Creek    2010      14          GE Wind   
54362      ID                    Yahoo Creek    2010      14          GE Wind   
55167      TX         unknown El Paso County    2015       2          GE Wind   
55531      AK  unknown Kotzebue Wind Project    2003       2  Seaforth Energy   

           t_model  t_cap  

In [109]:
df_wind_short2.to_csv('wind_usa_cleaned_st2.csv', index=False)